In [ ]:
!pip3 install -U langchain-community

In [ ]:
!pip3 install youtube-transcript-api

In [ ]:
import subprocess
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI as LOpenAI
from langchain.document_loaders import YoutubeLoader
import os
from dotenv import load_dotenv

load_dotenv('/workspace/repos/agentic-ai/.env')

model_name, ctx_len = "gpt-4o", 12000
# model_name, ctx_len = "deepseek-r1:32b", 128000
# model_name, ctx_len = "hermes3:8b", 128000

if model_name == "gpt-4o":
    openai_api_key = os.environ.get("OPENAI_KEY")
    # os.environ["OPENAI_API_KEY"] = openai_api_key
    llm = LOpenAI(model=model_name, max_tokens=8000, api_key=openai_api_key)
else:
    try: 
        print("Pulling Ollama model...")
        sub_out = subprocess.run(['ollama', 'pull', model_name], capture_output=True, text=True)
    except Exception as e: 
        print(f"Error pulling model: Is the Ollama server running?\n{e}")
    
    addtion_kwargs = {"max_new_tokens": 10000}
    # system_prompt = "You are an expert at answering questions about rules and regulations regarding Title 17—Commodity and Securities Exchanges: CHAPTER II—SECURITIES AND EXCHANGE COMMISSION. Please provide a summary of the following text, and cite any sections, rules, acts or laws (e.g. § 230.503, § 240.13a-15, Act (15 U.S.C. 781), Investment Company Act of 1940) from context that support the answer. Be detailed in your response."
    llm = Ollama(model=model_name, url="http://127.0.0.1:11434", context_window=ctx_len, model_type="chat", is_function_calling_model=False, 
                request_timeout=1000.0, additional_kwargs=addtion_kwargs) # system_prompt=system_prompt
    print(llm.metadata)


In [ ]:
url = 'https://www.youtube.com/watch?v=f6BnBvtolAw&authuser=4'

loader = YoutubeLoader.from_youtube_url(url)

documents_impact = loader.load()

In [ ]:
url = 'https://www.youtube.com/watch?v=BwoK2H8T_uU' # AI: The Ulitmate Human Invention?

loader = YoutubeLoader.from_youtube_url(url)

documents_ultimate= loader.load()

In [ ]:
print(documents_impact[0].page_content)

In [ ]:
from llama_index.core.response_synthesizers import TreeSummarize
import nest_asyncio
nest_asyncio.apply()

summarizer = TreeSummarize(llm=llm, verbose=True)
# prompt_summary = "You are a professional executive of AlphaTrAI. Your job is to summarize this text in great detail from a video transcription. The summary will be distributed to investors and stakeholders, so give a lot of details and examples from the transcription."
prompt_summary = """You will be given a video transcription. Your job is to summarize the text in great detail. The summary will be used for background on the guest speaker, so give a lot of details and examples from the transcription."""

response = await summarizer.aget_response(prompt_summary, [documents_impact[0].page_content])

In [ ]:
print(response)

In [ ]:
questions_prompt = """You will be given a video transcription, where the guest's name is Dan Tapiero. Your job is to come up with questions that you would ask Dan Tapiero based on the video transcription. The questions should be insightful and thought-provoking. Be sure to ask questions that will help you understand the speaker's perspective and opinions on the topic.

VIDEO TRANSCRIPTION:
{video_trascription}

QUESTIONS TO ASK DAN IN AN INTERVIEW:
"""

response = llm.complete(questions_prompt.format(video_trascription=documents_impact[0].page_content))

In [ ]:
print(response.text)

In [ ]:
bio = "Dan Tapiero is a renowned macro-focused investor with over 25 years of investing and entrepreneurial experience. He is the Founder, CEO and CIO of 1Roundtable Partners (1RT) and 10T Holdings (10T), which collectively manage over $1.4 billion of investments in 24 active companies across the Digital Asset Ecosystem (DAE). 1RT is a growth equity fund focused on mid-to-late stage companies in the DAE. Dan serves on several boards including Ledger, LEDN, Futureverse, and Quicknode. Before founding 1RT and 10T, Dan was Managing Partner of DTAP Capital Advisors, a global macro investment fund that he founded in 2003. He is Co-Founder of Gold Bullion International (GBI), a physical precious metals platform for the wealth management industry. GBI expanded into the cryptocurrency business through its integration with Uphold (fka Bitreserve) in 2014. Prior to GBI, he was formerly Co-Founder of the Agricultural Company of America (AGCoA), one of the largest farmland REITs in the U.S. at the time of its sale to the Canadian Pension Plan Investment Board (CPPIB) in 2013. Previously, from 1992-2012, Dan was a macro-focused portfolio manager and analyst at Duquesne Capital Management, SAC Capital Advisors, Tiger Management and Steinhardt Management Co. Dan graduated Phi Beta Kappa from Brown University, where he earned his B.A.'90 and M.A.'91 in European History. Dan has been on the Board of Trustees of the Lawrenceville School for the past 15 years and currently leads the Investment Committee for its endowment."

prompt = """You will be interviewing Dan Tapiero. You will be given Dan Tapiero's biography. Based on the biography, you will come up with questions that you would ask Dan Tapiero in an interview. The questions should be insightful and thought-provoking. Be sure to ask questions that will help you understand the speaker's perspective and opinions on the topic.

DAN TAPIERO'S BIOGRAPHY:
{biography}

QUESTIONS TO ASK DAN IN AN INTERVIEW:"""

In [ ]:
response = llm.complete(prompt.format(biography=bio))


In [ ]:
print(response.text)

In [ ]:
print(documents_ultimate[0].page_content)

In [ ]:
from llama_index.core.response_synthesizers import TreeSummarize
import nest_asyncio
nest_asyncio.apply()

summarizer = TreeSummarize(llm=llm, verbose=True)
# prompt_summary = "You are a professional executive of AlphaTrAI. Your job is to summarize this text in great detail from a video transcription. The summary will be distributed to investors and stakeholders, so give a lot of details and examples from the transcription."
prompt_summary = f"""Summarize and compare the two following transcribed texts so that it will be useful to consultant companies who want to provide AI solutions to the business presenting in the text. 
The first text is from the video "The Impact of Artificial Intelligence - Verify You're Human; Ask Me Anything" and the second text is from the video "AI: The Ulitmate Human Invention?".
Highlight the similarities between the two texts.
Be detailed in your response and provide examples from the text."""

response = await summarizer.aget_response(prompt_summary, [documents_impact[0].page_content, documents_ultimate[0].page_content])

In [ ]:
print(response)

In [ ]:
# Summarize the following transcribed text so that it will be useful to consultant companies who want to provide AI solutions to the business presenting in the text. 
# Be detailed in your response and provide examples from the text.

# The Impact of Artificial Intelligence - Verify You're Human; Ask Me Anything
# The webinar, led by Mark McCarron and Grant Rodden from West College, discusses the evolution and impact of artificial intelligence (AI) on various industries, with a focus on investment and wealth management. The conversation begins with a historical overview of AI, highlighting milestones such as the Turing Test, the development of IBM's Deep Blue, and the introduction of Siri and Watson. The discussion then shifts to the recent advancements in generative AI, exemplified by OpenAI's ChatGPT, which can train itself using large datasets and perform tasks like converting text to images or videos.

# The presenters explore the potential applications of AI across different sectors. In the automotive industry, AI is used in self-driving technologies, as seen in Tesla's integration of AI for autonomous driving. In marketing, AI can generate new ad campaigns, while in healthcare, it aids in drug discovery by quickly analyzing various scenarios. The entertainment industry uses AI to create new content by combining historical and current actor performances. In wealth management, AI is beginning to impact how financial advice is delivered, although it currently lacks the ability to provide personalized advice without human input.

# The webinar also addresses the challenges of implementing AI, such as the need for significant computing power and high-quality data. The cost and accessibility of data licenses are also discussed, along with the importance of policy and regulation to manage AI's unchecked use.

# From an investment perspective, the presenters note the significant market impact of AI, with companies like Nvidia, Meta, and Google leading the charge. Nvidia, in particular, is highlighted for its dominance in the AI chip market, driving substantial market gains. However, the high valuation of such companies is noted as a potential risk.

# For consultant companies looking to provide AI solutions, the key takeaways include the importance of understanding AI's diverse applications across industries, the need for robust computing infrastructure, and the necessity of high-quality data. Additionally, consultants should be aware of the regulatory landscape and the potential market volatility associated with AI investments.

# AI: The Ulitmate Human Invention?
# The discussion between Carrie Westcott and Mark McCarron highlights the transformative impact of AI, particularly generative AI, on various industries. AI is enabling machines to contribute creatively, producing original content like text, images, and music, and is seen as a revolutionary technological development. The conversation emphasizes the importance of businesses harnessing AI's power while managing risks related to data privacy and ethical use.

# Key points for consultant companies include:

# 1. **AI Adoption and Applications**: AI is being widely adopted across industries for automating processes, monitoring systems, and enhancing customer service. Examples include industrial manufacturing and document flow management. AI chatbots are commonly used for customer interactions.

# 2. **Workplace Integration**: A significant portion of the workforce is already using AI, with many bringing their own AI tools to work. This trend spans across generations, not just younger employees. Companies are increasingly requiring AI skills from new hires.

# 3. **Ethical and Trustworthy AI**: There is a strong emphasis on developing transparent and ethical AI practices. Data privacy is a major concern, and companies are focusing on ensuring AI solutions are unbiased and explainable.

# 4. **Leading Companies and Countries**: Major players in the AI space include Nvidia, Microsoft, Alphabet, and others. These companies are at the forefront of AI technology development. The US, China, and India are leading in AI deployment, with cultural and operational differences influencing their approaches.

# 5. **Investment Opportunities**: AI is pervasive in investment portfolios, with direct exposure through companies like Microsoft and Nvidia, and indirect exposure through firms utilizing AI to enhance operations, such as healthcare providers and financial institutions.

# Consultant companies can leverage these insights to tailor AI solutions that address specific business needs, ensure ethical practices, and capitalize on the growing demand for AI expertise. They can also guide businesses in integrating AI into their operations to improve efficiency and profitability.

In [ ]:
# The two transcribed texts, "The Impact of Artificial Intelligence - Verify You're Human; Ask Me Anything" and "AI: The Ultimate Human Invention?" both explore the transformative potential of artificial intelligence (AI) across various industries, highlighting its historical evolution, current applications, and future implications. 

# **Similarities:**
# 1. **Historical Context and Evolution:** Both texts provide a timeline of AI's development, mentioning key milestones such as the Turing Test, the creation of Deep Blue, and the emergence of generative AI models like ChatGPT. This historical perspective sets the stage for understanding AI's current capabilities and future potential.

# 2. **Generative AI and Its Applications:** Both texts emphasize the role of generative AI in revolutionizing industries by enabling machines to perform tasks traditionally done by humans. Examples include creating new content, automating processes, and enhancing decision-making. The texts mention applications in sectors like healthcare, automotive, and entertainment.

# 3. **Market Impact and Investment Opportunities:** Both texts discuss the significant impact of AI on financial markets, highlighting companies like Nvidia, Microsoft, and Google as key players benefiting from AI advancements. They note the surge in stock prices for these companies, driven by investor enthusiasm for AI technologies.

# 4. **Challenges and Ethical Considerations:** Both texts address the challenges associated with AI, such as the need for high-quality data, computing power, and ethical considerations. They emphasize the importance of developing trustworthy AI practices to ensure data privacy and unbiased decision-making.

# 5. **Future Outlook and Workforce Implications:** Both texts speculate on AI's future, considering its potential to drive productivity and efficiency while also raising concerns about job displacement. They highlight the need for AI skills in the workforce and the potential for AI to create new job opportunities.

# **Examples from the Texts:**
# - The first text discusses AI's impact on businesses and the economy, with specific use cases in marketing, automotive, and healthcare. It also highlights the investment case for AI, noting the significant returns for companies like Nvidia.
# - The second text provides a broader view of AI's adoption across industries, citing surveys that show widespread use of AI in automating processes and improving efficiency. It also discusses the ethical challenges and the need for transparent AI practices.

# For consultant companies looking to provide AI solutions, these texts offer valuable insights into the current landscape of AI, the industries most affected, and the key players driving innovation. Understanding these aspects can help consultants tailor their services to meet the specific needs of businesses looking to leverage AI technologies.

In [ ]:
!pip3 install moviepy openai python-dotenv pydub pytubefix openai-whisper llama-index llama-index-llms-openai
!apt-get install ffmpeg -y

In [ ]:
import os, pickle, gc, subprocess
from dotenv import load_dotenv
from video_transcription import split_audio_into_chunks, video_to_audio, transcribe_audio_chunks, get_transcription_model, download_video
from llama_index.core import Document
# import torch

load_dotenv("/workspace/repos/agentic-ai/.env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
video_path = 'https://www.youtube.com/watch?v=k93EoHAg7BY' # The Impact of Artificial Intelligence - Verify You're Human; Ask Me Anything
# video_path = 'https://www.youtube.com/watch?v=BwoK2H8T_uU' # AI: The Ulitmate Human Invention?
video_output = "/workspace/data/ai_impact.mp4" 
audio_output_path="/workspace/data/ai_impact.mp3"
transcribe_output_dir="/workspace/data"
file_save_path="/workspace/data/ai_impact_transcription.pkl"
if not os.path.exists(file_save_path):
    print("Loading transcription model...")
    transcribe_model = get_transcription_model(open_source_model=True)

    print("Processing video...")
    if 'youtube' in video_path:
        print("   Downloading video from youtube...")
        download_video(video_url=video_path, video_output_path=video_output)
    else:
        video_output = video_path
    
    print("Converting video to audio...")
    video_to_audio(video_path=video_output, audio_output_path=audio_output_path)

    print("Splitting audio into chunks...")
    split_audio_into_chunks(audio_output_path=audio_output_path, transcribe_output_dir=transcribe_output_dir, max_chunk_size_mb=22)

    print("Transcribing audio chunks...")
    documents = transcribe_audio_chunks(model=transcribe_model, chunk_dir="/workspace/data", file_save_path=file_save_path)

    del transcribe_model
    # torch.cuda.empty_cache()
    gc.collect()
else:
    with open(file_save_path, 'rb') as f:
        documents = pickle.load(f)
    
if isinstance(documents[0], dict):
    documents = [Document(text=chunk["text"]) for chunk in documents]

full_doc = " ".join([doc.text for doc in documents])


In [ ]:
from moviepy.editor import VideoFileClip
clip = VideoFileClip(video_output)
audio = clip.audio
audio.write_audiofile(audio_output_path)

In [ ]:
clip?

In [ ]:
download_video(video_url=video_path, audio_output_path=audio_output_path)

In [ ]:
import pytubefix as pt

yt = pt.YouTube(video_path)
stream = yt.streams.filter(only_video=True).first()
stream.download(output_path=os.path.dirname(audio_output_path), filename=os.path.basename(audio_output_path))

In [ ]:
import pytubefix as pt
yt = pt.YouTube(video_path)

In [ ]:
xyt